In [1]:
import os
import torch
import random
import math
from torch import nn
from torch.nn.modules import activation
import torch.nn.functional as F
import torchvision
import shutil
import numpy as np
from PIL import Image
import time
from tqdm.notebook import tqdm
from sklearn.metrics import confusion_matrix
import TAR_transformer

C:\Users\GillA\.conda\envs\Aseem\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.1
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
def correct_sizes(sizes):
	corrected_sizes = [s if s % 2 != 0 else s - 1 for s in sizes]
	return corrected_sizes


def pass_through(X):
	return X


class Inception(nn.Module):
	def __init__(self, in_channels, n_filters, kernel_sizes=[9, 19, 39], bottleneck_channels=32, activation=nn.ReLU(), return_indices=False):
		"""
		: param in_channels				Number of input channels (input features)
		: param n_filters				Number of filters per convolution layer => out_channels = 4*n_filters
		: param kernel_sizes			List of kernel sizes for each convolution.
										Each kernel size must be odd number that meets -> "kernel_size % 2 !=0".
										This is nessesery because of padding size.
										For correction of kernel_sizes use function "correct_sizes". 
		: param bottleneck_channels		Number of output channels in bottleneck. 
										Bottleneck wont be used if nuber of in_channels is equal to 1.
		: param activation				Activation function for output tensor (nn.ReLU()). 
		: param return_indices			Indices are needed only if we want to create decoder with InceptionTranspose with MaxUnpool1d. 
		"""
		super(Inception, self).__init__()
		self.return_indices=return_indices
		if in_channels > 1:
			self.bottleneck = nn.Conv1d(
								in_channels=in_channels, 
								out_channels=bottleneck_channels, 
								kernel_size=1, 
								stride=1, 
								bias=False
								)
		else:
			self.bottleneck = pass_through
			bottleneck_channels = 1

		self.conv_from_bottleneck_1 = nn.Conv1d(
										in_channels=bottleneck_channels, 
										out_channels=n_filters, 
										kernel_size=kernel_sizes[0], 
										stride=1, 
										padding=kernel_sizes[0]//2, 
										bias=False
										)
		self.conv_from_bottleneck_2 = nn.Conv1d(
										in_channels=bottleneck_channels, 
										out_channels=n_filters, 
										kernel_size=kernel_sizes[1], 
										stride=1, 
										padding=kernel_sizes[1]//2, 
										bias=False
										)
		self.conv_from_bottleneck_3 = nn.Conv1d(
										in_channels=bottleneck_channels, 
										out_channels=n_filters, 
										kernel_size=kernel_sizes[2], 
										stride=1, 
										padding=kernel_sizes[2]//2, 
										bias=False
										)
		self.max_pool = nn.MaxPool1d(kernel_size=3, stride=1, padding=1, return_indices=return_indices)
		self.conv_from_maxpool = nn.Conv1d(
									in_channels=in_channels, 
									out_channels=n_filters, 
									kernel_size=1, 
									stride=1,
									padding=0, 
									bias=False
									)
		self.batch_norm = nn.BatchNorm1d(num_features=4*n_filters)
		self.activation = activation

	def forward(self, X):
		# step 1
		Z_bottleneck = self.bottleneck(X)
		if self.return_indices:
			Z_maxpool, indices = self.max_pool(X)
		else:
			Z_maxpool = self.max_pool(X)
		# step 2
		Z1 = self.conv_from_bottleneck_1(Z_bottleneck)
		Z2 = self.conv_from_bottleneck_2(Z_bottleneck)
		Z3 = self.conv_from_bottleneck_3(Z_bottleneck)
		Z4 = self.conv_from_maxpool(Z_maxpool)
		# step 3 
		Z = torch.cat([Z1, Z2, Z3, Z4], axis=1)
		Z = self.activation(self.batch_norm(Z))
		if self.return_indices:
			return Z, indices
		else:
			return Z


class InceptionBlock(nn.Module):
	def __init__(self, in_channels, n_filters=32, kernel_sizes=[9,19,39], bottleneck_channels=32, use_residual=True, activation=nn.ReLU(), return_indices=False):
		super(InceptionBlock, self).__init__()
		self.use_residual = use_residual
		self.return_indices = return_indices
		self.activation = activation
		self.inception_1 = Inception(
							in_channels=in_channels,
							n_filters=n_filters,
							kernel_sizes=kernel_sizes,
							bottleneck_channels=bottleneck_channels,
							activation=activation,
							return_indices=return_indices
							)
		self.inception_2 = Inception(
							in_channels=4*n_filters,
							n_filters=n_filters,
							kernel_sizes=kernel_sizes,
							bottleneck_channels=bottleneck_channels,
							activation=activation,
							return_indices=return_indices
							)
		self.inception_3 = Inception(
							in_channels=4*n_filters,
							n_filters=n_filters,
							kernel_sizes=kernel_sizes,
							bottleneck_channels=bottleneck_channels,
							activation=activation,
							return_indices=return_indices
							)	
		if self.use_residual:
			self.residual = nn.Sequential(
								nn.Conv1d(
									in_channels=in_channels, 
									out_channels=4*n_filters, 
									kernel_size=1,
									stride=1,
									padding=0
									),
								nn.BatchNorm1d(
									num_features=4*n_filters
									)
								)

	def forward(self, X):
		if self.return_indices:
			Z, i1 = self.inception_1(X)
			Z, i2 = self.inception_2(Z)
			Z, i3 = self.inception_3(Z)
		else:
			Z = self.inception_1(X)
			Z = self.inception_2(Z)
			Z = self.inception_3(Z)
		if self.use_residual:
			Z = Z + self.residual(X)
			Z = self.activation(Z)
		if self.return_indices:
			return Z,[i1, i2, i3]
		else:
			return Z



class InceptionTranspose(nn.Module):
	def __init__(self, in_channels, out_channels, kernel_sizes=[9, 19, 39], bottleneck_channels=32, activation=nn.ReLU()):
		"""
		: param in_channels				Number of input channels (input features)
		: param n_filters				Number of filters per convolution layer => out_channels = 4*n_filters
		: param kernel_sizes			List of kernel sizes for each convolution.
										Each kernel size must be odd number that meets -> "kernel_size % 2 !=0".
										This is nessesery because of padding size.
										For correction of kernel_sizes use function "correct_sizes". 
		: param bottleneck_channels		Number of output channels in bottleneck. 
										Bottleneck wont be used if nuber of in_channels is equal to 1.
		: param activation				Activation function for output tensor (nn.ReLU()). 
		"""
		super(InceptionTranspose, self).__init__()
		self.activation = activation
		self.conv_to_bottleneck_1 = nn.ConvTranspose1d(
										in_channels=in_channels, 
										out_channels=bottleneck_channels, 
										kernel_size=kernel_sizes[0], 
										stride=1, 
										padding=kernel_sizes[0]//2, 
										bias=False
										)
		self.conv_to_bottleneck_2 = nn.ConvTranspose1d(
										in_channels=in_channels, 
										out_channels=bottleneck_channels, 
										kernel_size=kernel_sizes[1], 
										stride=1, 
										padding=kernel_sizes[1]//2, 
										bias=False
										)
		self.conv_to_bottleneck_3 = nn.ConvTranspose1d(
										in_channels=in_channels, 
										out_channels=bottleneck_channels, 
										kernel_size=kernel_sizes[2], 
										stride=1, 
										padding=kernel_sizes[2]//2, 
										bias=False
										)
		self.conv_to_maxpool = nn.Conv1d(
									in_channels=in_channels, 
									out_channels=out_channels, 
									kernel_size=1, 
									stride=1,
									padding=0, 
									bias=False
									)
		self.max_unpool = nn.MaxUnpool1d(kernel_size=3, stride=1, padding=1)
		self.bottleneck = nn.Conv1d(
								in_channels=3*bottleneck_channels, 
								out_channels=out_channels, 
								kernel_size=1, 
								stride=1, 
								bias=False
								)
		self.batch_norm = nn.BatchNorm1d(num_features=out_channels)

		def forward(self, X, indices):
			Z1 = self.conv_to_bottleneck_1(X)
			Z2 = self.conv_to_bottleneck_2(X)
			Z3 = self.conv_to_bottleneck_3(X)
			Z4 = self.conv_to_maxpool(X)

			Z = torch.cat([Z1, Z2, Z3], axis=1)
			MUP = self.max_unpool(Z4, indices)
			BN = self.bottleneck(Z)
			# another possibility insted of sum BN and MUP is adding 2nd bottleneck transposed convolution
			
			return self.activation(self.batch_norm(BN + MUP))


class InceptionTransposeBlock(nn.Module):
	def __init__(self, in_channels, out_channels=32, kernel_sizes=[9,19,39], bottleneck_channels=32, use_residual=True, activation=nn.ReLU()):
		super(InceptionTransposeBlock, self).__init__()
		self.use_residual = use_residual
		self.activation = activation
		self.inception_1 = InceptionTranspose(
							in_channels=in_channels,
							out_channels=in_channels,
							kernel_sizes=kernel_sizes,
							bottleneck_channels=bottleneck_channels,
							activation=activation
							)
		self.inception_2 = InceptionTranspose(
							in_channels=in_channels,
							out_channels=in_channels,
							kernel_sizes=kernel_sizes,
							bottleneck_channels=bottleneck_channels,
							activation=activation
							)
		self.inception_3 = InceptionTranspose(
							in_channels=in_channels,
							out_channels=out_channels,
							kernel_sizes=kernel_sizes,
							bottleneck_channels=bottleneck_channels,
							activation=activation
							)	
		if self.use_residual:
			self.residual = nn.Sequential(
								nn.ConvTranspose1d(
									in_channels=in_channels, 
									out_channels=out_channels, 
									kernel_size=1,
									stride=1,
									padding=0
									),
								nn.BatchNorm1d(
									num_features=out_channels
									)
								)

	def forward(self, X, indices):
		assert len(indices)==3
		Z = self.inception_1(X, indices[2])
		Z = self.inception_2(Z, indices[1])
		Z = self.inception_3(Z, indices[0])
		if self.use_residual:
			Z = Z + self.residual(X)
			Z = self.activation(Z)
		return Z

class Flatten(nn.Module):
	def __init__(self, out_features):
		super(Flatten, self).__init__()
		self.output_dim = out_features

	def forward(self, x):
		return x.view(-1, self.output_dim)
    
class Reshape(nn.Module):
	def __init__(self, out_shape):
		super(Reshape, self).__init__()
		self.out_shape = out_shape

	def forward(self, x):
		return x.view(-1, *self.out_shape)

In [3]:
class Inception_with_auxillary(nn.Module):
    def __init__(self):
        super(Inception_with_auxillary, self).__init__()
        
        self.reshape = Reshape(out_shape=(56,1500))
        self.block_i = InceptionBlock(
            in_channels=56, 
            n_filters=32, 
            kernel_sizes=[5, 11, 23],
            bottleneck_channels=32,
            use_residual=True,
            activation=nn.ReLU())
        
        self.block = InceptionBlock(
            in_channels=32*4, 
            n_filters=32, 
            kernel_sizes=[5, 11, 23],
            bottleneck_channels=32,
            use_residual=True,
            activation=nn.ReLU())
        
        self.auxillary_out = nn.Sequential(
                    nn.AdaptiveAvgPool1d(output_size=1),
                    Flatten(out_features=32*4*1),
                    nn.Linear(in_features=4*32*1, out_features=3))
        
    def forward(self,input_mat):
        # Initial Layers
        resized_input_mat = self.reshape(input_mat)
        Inception_out_1 = self.block_i(resized_input_mat)
        Inception_out_2 = self.block(Inception_out_1)

        # Auxillary 1
        aux_1 = self.auxillary_out(Inception_out_2)

        # Deep Blocks 1
        Inception_out_3 = self.block(Inception_out_2)
        Inception_out_4 = self.block(Inception_out_3)

        # Auxillary 2
        aux_2 = self.auxillary_out(Inception_out_2)

        # Deep Blocks 1
        Inception_out_5 = self.block(Inception_out_4)
        Inception_out_6 = self.block(Inception_out_5)

        # Final Out
        main = self.auxillary_out(Inception_out_6)

#         aux_1 = nn.Softmax(aux_1)
#         aux_2 = nn.Softmax(aux_2)
#         main = nn.Softmax(main)

        out_arr = [aux_1, aux_2, main]
    
        return(out_arr)
        

In [4]:
class MSCB(nn.Module):
    def __init__(self, small_kernel, medium_kernel, large_kernel, num_filters):
        super(MSCB, self).__init__()
        self.name = "MSCB"

        # Define Small Path
        self.convS = nn.Conv1d(in_channels = 56, out_channels = num_filters, kernel_size = small_kernel, padding = 'same')
        self.MPoolS = nn.MaxPool1d(kernel_size = small_kernel, stride = 5, padding = int(small_kernel/2 - 1))
        
        # Define Medium Path
        self.convM = nn.Conv1d(in_channels = 56, out_channels = num_filters, kernel_size = medium_kernel, padding = 'same')
        self.MPoolM = nn.MaxPool1d(kernel_size = medium_kernel, stride = 5, padding = int(medium_kernel/2 - 1))
        
        # Define Large Path
        self.convL = nn.Conv1d(in_channels = 56, out_channels = num_filters, kernel_size = large_kernel, padding = 'same')
        self.MPoolL = nn.MaxPool1d(kernel_size = large_kernel, stride = 5, padding = int(large_kernel/2 - 1))
        
        #
        self.MPool = nn.MaxPool1d(kernel_size = 3, stride = 5)
        self.conv = nn.Conv1d(in_channels = 56, out_channels = 128, kernel_size = 24, padding = 'same')
        
        #
        self.conv2 = nn.Conv1d(in_channels = 3*num_filters + 128, out_channels = 64, kernel_size = 112, padding = 'same')
        self.MPool2 = nn.MaxPool1d(kernel_size = 3, stride = 5)
        self.fc1 = nn.Linear(in_features = 5760, out_features = 400)
        self.Dropout = nn.Dropout(0.5)
        self.fc2 = nn.Linear(in_features = 400, out_features = 1024)
        self.fc3 = nn.Linear(in_features = 1024, out_features = 3)

    def forward(self, x):
        # Feature Learning Head
        
        x = torch.moveaxis(x,2,1)
        x = torch.tensor(x, dtype=torch.float32)

        x_S = self.MPoolS(F.relu(self.convS(x)))
        x_M = self.MPoolM(F.relu(self.convM(x)))
        x_L = self.MPoolL(F.relu(self.convL(x)))
        x_O = F.relu(self.conv(self.MPool(x)))
        
#         print(x_S.shape)
#         print(x_M.shape)
#         print(x_L.shape)
#         print(x_O.shape)
        
        #
        x = torch.cat((x_S,x_M,x_L,x_O),1)
        x = self.MPool2(F.relu(self.conv2(x)))

        # Flattening
        x = x.view(-1,5760)

        #Classification Head
        x = F.relu(self.fc1((x)))
        x = self.Dropout(x)
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x



In [5]:
### FUNCTION FOR SETTING UP DATA LOADERS
def prepare_valtest_dataset(data_dir, batch_size =  64, num_workers = 0):

    # Specific where the folder containing the images is for the specific dataset
    # This function contains sub-directories each containing all images in a 
    # single class
    data = torchvision.datasets.DatasetFolder(data_dir, loader = torch.load, extensions = ".pt")#, transform=data_transform)
    

    # prepare data loaders
    loader = torch.utils.data.DataLoader(data, batch_size=batch_size, 
                                            num_workers=num_workers, shuffle=True)

    return loader
### FUNCTION FOR SETTING UP DATA LOADERS
def prepare_train_dataset(data_dir, batch_size =  64, num_workers = 0):

    # Specific where the folder containing the images is for the specific dataset
    # This function contains sub-directories each containing all images in a 
    # single class
    data1 = torchvision.datasets.DatasetFolder(data_dir[0], loader = torch.load, extensions = ".pt")
    data2 = torchvision.datasets.DatasetFolder(data_dir[1], loader = torch.load, extensions = ".pt")
    train_data = torch.utils.data.ConcatDataset((data1,data2))

    # prepare data loaders
    loader = torch.utils.data.DataLoader(train_data, batch_size=batch_size, 
                                            num_workers=num_workers, shuffle=True)

    return loader

In [6]:
# base_dir = "M:\Peripheral Nerve Studies\MCC Projects\Aseem G\Models\Pytorch\Data\Spike Firing Rate\Minimum Dataset\Debug_Rat\\"
# fold1_dir = base_dir + "Fold1"
# fold2_dir = base_dir + "Fold2"
# fold3_dir = base_dir + "Fold3"
# test_dir = base_dir + "Test"

def three_fold_cross_sets(base_dir,fold1_dir,fold2_dir,fold3_dir,test_dir, batch_size =  64, num_workers = 0):
    #Fold 1
    train_set_1 = prepare_train_dataset([fold1_dir,fold2_dir], batch_size, num_workers)
    valid_set_1 = prepare_valtest_dataset(fold3_dir, batch_size, num_workers)

    #Fold 2

    train_set_2 = prepare_train_dataset([fold1_dir,fold3_dir], batch_size, num_workers)
    valid_set_2 = prepare_valtest_dataset(fold2_dir, batch_size, num_workers)

    #Fold 3
    train_set_3 = prepare_train_dataset([fold2_dir,fold3_dir],batch_size, num_workers)
    valid_set_3 = prepare_valtest_dataset(fold1_dir, batch_size, num_workers)

    test_set = prepare_valtest_dataset(test_dir, batch_size, num_workers)
    
    return train_set_1, valid_set_1, train_set_2, valid_set_2, train_set_3, valid_set_3, test_set

In [7]:
def accuracy(array):
    total_samples = np.sum(array)/1.
    correct = np.sum(np.multiply(array,np.eye(3)))

    accuracy = round(correct/total_samples*100,2)
    return(accuracy)
def recall(array, index):
    num = array[index,index]
    den = np.sum(array[index,:])
    
    return(num/den)

def precision(array, index):
    num = array[index,index]
    den = np.sum(array[:,index])
    
    return(num/den)
def macro_f1(array):
    recall_arr = []
    precision_arr = []

    for i in range(3):
        recall_arr.append(recall(array, i))
        precision_arr.append(precision(array, i))
        
    per_class_f1 = []
    for i in range(3):
        per_class_f1.append((2*recall_arr[i]*precision_arr[i])/(recall_arr[i] + precision_arr[i]))
        
    f1 = sum(per_class_f1)/3
    return f1

In [8]:
torch.manual_seed(1000)

In [9]:
# # 1500 in


# class MSCB(nn.Module):
#     def __init__(self, small_kernel, medium_kernel, large_kernel, num_filters):
#         super(MSCB, self).__init__()
#         self.name = "MSCB"

#         # Define Small Path
#         self.convS = nn.Conv1d(in_channels = 56, out_channels = num_filters, kernel_size = small_kernel, padding = 'same')
#         self.MPoolS = nn.MaxPool1d(kernel_size = small_kernel, stride = 5, padding = int(small_kernel/2 - 1))
        
#         # Define Medium Path
#         self.convM = nn.Conv1d(in_channels = 56, out_channels = num_filters, kernel_size = medium_kernel, padding = 'same')
#         self.MPoolM = nn.MaxPool1d(kernel_size = medium_kernel, stride = 5, padding = int(medium_kernel/2 - 1))
        
#         # Define Large Path
#         self.convL = nn.Conv1d(in_channels = 56, out_channels = num_filters, kernel_size = large_kernel, padding = 'same')
#         self.MPoolL = nn.MaxPool1d(kernel_size = large_kernel, stride = 5, padding = int(large_kernel/2 - 1))
        
#         #
#         self.MPool = nn.MaxPool1d(kernel_size = 3, stride = 5)
#         self.conv = nn.Conv1d(in_channels = 56, out_channels = 128, kernel_size = 24, padding = 'same')
        
#         #
#         self.conv2 = nn.Conv1d(in_channels = 3*num_filters + 128, out_channels = 64, kernel_size = 112, padding = 'same')
#         self.MPool2 = nn.MaxPool1d(kernel_size = 3, stride = 5)
#         self.fc1 = nn.Linear(in_features = 3840, out_features = 400)
#         self.Dropout = nn.Dropout(0.5)
#         self.fc2 = nn.Linear(in_features = 400, out_features = 1024)
#         self.fc3 = nn.Linear(in_features = 1024, out_features = 3)

#     def forward(self, x):
#         # Feature Learning Head
        
#         x = torch.moveaxis(x,2,1)
#         x = torch.tensor(x, dtype=torch.float32)

#         x_S = self.MPoolS(F.relu(self.convS(x)))
#         x_M = self.MPoolM(F.relu(self.convM(x)))
#         x_L = self.MPoolL(F.relu(self.convL(x)))
#         x_O = F.relu(self.conv(self.MPool(x)))
        
# #         print(x_S.shape)
# #         print(x_M.shape)
# #         print(x_L.shape)
# #         print(x_O.shape)
        
#         #
#         x = torch.cat((x_S,x_M,x_L,x_O),1)
#         x = self.MPool2(F.relu(self.conv2(x)))
        
#         # Flattening
#         x = x.view(-1,3840)

#         #Classification Head
#         x = F.relu(self.fc1((x)))
#         x = self.Dropout(x)
#         x = F.relu(self.fc2(x))
#         x = self.fc3(x)
#         return x



In [10]:
def correct_sizes(sizes):
	corrected_sizes = [s if s % 2 != 0 else s - 1 for s in sizes]
	return corrected_sizes


def pass_through(X):
	return X


class Inception(nn.Module):
	def __init__(self, in_channels, n_filters, kernel_sizes=[9, 19, 39], bottleneck_channels=32, activation=nn.ReLU(), return_indices=False):
		"""
		: param in_channels				Number of input channels (input features)
		: param n_filters				Number of filters per convolution layer => out_channels = 4*n_filters
		: param kernel_sizes			List of kernel sizes for each convolution.
										Each kernel size must be odd number that meets -> "kernel_size % 2 !=0".
										This is nessesery because of padding size.
										For correction of kernel_sizes use function "correct_sizes". 
		: param bottleneck_channels		Number of output channels in bottleneck. 
										Bottleneck wont be used if nuber of in_channels is equal to 1.
		: param activation				Activation function for output tensor (nn.ReLU()). 
		: param return_indices			Indices are needed only if we want to create decoder with InceptionTranspose with MaxUnpool1d. 
		"""
		super(Inception, self).__init__()
		self.return_indices=return_indices
		if in_channels > 1:
			self.bottleneck = nn.Conv1d(
								in_channels=in_channels, 
								out_channels=bottleneck_channels, 
								kernel_size=1, 
								stride=1, 
								bias=False
								)
		else:
			self.bottleneck = pass_through
			bottleneck_channels = 1

		self.conv_from_bottleneck_1 = nn.Conv1d(
										in_channels=bottleneck_channels, 
										out_channels=n_filters, 
										kernel_size=kernel_sizes[0], 
										stride=1, 
										padding=kernel_sizes[0]//2, 
										bias=False
										)
		self.conv_from_bottleneck_2 = nn.Conv1d(
										in_channels=bottleneck_channels, 
										out_channels=n_filters, 
										kernel_size=kernel_sizes[1], 
										stride=1, 
										padding=kernel_sizes[1]//2, 
										bias=False
										)
		self.conv_from_bottleneck_3 = nn.Conv1d(
										in_channels=bottleneck_channels, 
										out_channels=n_filters, 
										kernel_size=kernel_sizes[2], 
										stride=1, 
										padding=kernel_sizes[2]//2, 
										bias=False
										)
		self.max_pool = nn.MaxPool1d(kernel_size=3, stride=1, padding=1, return_indices=return_indices)
		self.conv_from_maxpool = nn.Conv1d(
									in_channels=in_channels, 
									out_channels=n_filters, 
									kernel_size=1, 
									stride=1,
									padding=0, 
									bias=False
									)
		self.batch_norm = nn.BatchNorm1d(num_features=4*n_filters)
		self.activation = activation

	def forward(self, X):
		# step 1
		Z_bottleneck = self.bottleneck(X)
		if self.return_indices:
			Z_maxpool, indices = self.max_pool(X)
		else:
			Z_maxpool = self.max_pool(X)
		# step 2
		Z1 = self.conv_from_bottleneck_1(Z_bottleneck)
		Z2 = self.conv_from_bottleneck_2(Z_bottleneck)
		Z3 = self.conv_from_bottleneck_3(Z_bottleneck)
		Z4 = self.conv_from_maxpool(Z_maxpool)
		# step 3 
		Z = torch.cat([Z1, Z2, Z3, Z4], axis=1)
		Z = self.activation(self.batch_norm(Z))
# 		Z = self.activation(Z)
		if self.return_indices:
			return Z, indices
		else:
			return Z


class InceptionBlock(nn.Module):
	def __init__(self, in_channels, n_filters=32, kernel_sizes=[9,19,39], bottleneck_channels=32, use_residual=True, activation=nn.ReLU(), return_indices=False):
		super(InceptionBlock, self).__init__()
		self.use_residual = use_residual
		self.return_indices = return_indices
		self.activation = activation
		self.inception_1 = Inception(
							in_channels=in_channels,
							n_filters=n_filters,
							kernel_sizes=kernel_sizes,
							bottleneck_channels=bottleneck_channels,
							activation=activation,
							return_indices=return_indices
							)
		self.inception_2 = Inception(
							in_channels=4*n_filters,
							n_filters=n_filters,
							kernel_sizes=kernel_sizes,
							bottleneck_channels=bottleneck_channels,
							activation=activation,
							return_indices=return_indices
							)
		self.inception_3 = Inception(
							in_channels=4*n_filters,
							n_filters=n_filters,
							kernel_sizes=kernel_sizes,
							bottleneck_channels=bottleneck_channels,
							activation=activation,
							return_indices=return_indices
							)	
		if self.use_residual:
			self.residual = nn.Sequential(
								nn.Conv1d(
									in_channels=in_channels, 
									out_channels=4*n_filters, 
									kernel_size=1,
									stride=1,
									padding=0
									),
								nn.BatchNorm1d(
									num_features=4*n_filters
									)
								)

	def forward(self, X):
		if self.return_indices:
			Z, i1 = self.inception_1(X)
			Z, i2 = self.inception_2(Z)
			Z, i3 = self.inception_3(Z)
		else:
			Z = self.inception_1(X)
			Z = self.inception_2(Z)
			Z = self.inception_3(Z)
		if self.use_residual:
			Z = Z + self.residual(X)
			Z = self.activation(Z)
		if self.return_indices:
			return Z,[i1, i2, i3]
		else:
			return Z


class Flatten(nn.Module):
	def __init__(self, out_features):
		super(Flatten, self).__init__()
		self.output_dim = out_features

	def forward(self, x):
		return x.view(-1, self.output_dim)
    
class Reshape(nn.Module):
	def __init__(self, out_shape):
		super(Reshape, self).__init__()
		self.out_shape = out_shape

	def forward(self, x):
		return x.view(-1, *self.out_shape)

class Inception_with_auxillary(nn.Module):
    def __init__(self, n_filters, kernel_sizes, bottleneck_channels, GAPoutput_size, use_residual):
        super(Inception_with_auxillary, self).__init__()
        
        self.reshape = Reshape(out_shape=(56,1500))
        self.block_i = InceptionBlock(
            in_channels=56, 
            n_filters = n_filters, 
            kernel_sizes = kernel_sizes,
            bottleneck_channels = bottleneck_channels,
            use_residual = use_residual,
            activation=nn.ReLU())
        
        self.block = InceptionBlock(
            in_channels=n_filters*4, 
            n_filters = n_filters, 
            kernel_sizes = kernel_sizes,
            bottleneck_channels = bottleneck_channels,
            use_residual = use_residual,
            activation=nn.ReLU())

        self.MPoolM = nn.MaxPool1d(kernel_size = kernel_sizes[1], stride = 5, padding = int(kernel_sizes[1]/2 - 1)),

        
        self.alt_out = nn.Sequential(
            nn.MaxPool1d(kernel_size = 3, stride = 5),
            nn.Conv1d(in_channels = n_filters*4, out_channels = 64, kernel_size = 112, padding = 'same'),
            nn.MaxPool1d(kernel_size = 3, stride = 5),
            nn.Flatten(),
            nn.Linear(in_features = 3840, out_features = 400),
            nn.Dropout(0.5),
            nn.Linear(in_features = 400, out_features = 1024),
            nn.Linear(in_features = 1024, out_features = 3),
            nn.LogSoftmax(-1))
        
        
        self.auxillary_out = nn.Sequential(
            
                    nn.AdaptiveAvgPool1d(GAPoutput_size),
                    Flatten(out_features=n_filters*4*GAPoutput_size),
#                     nn.Dropout(0.5),
                    nn.Linear(in_features=4*n_filters*GAPoutput_size, out_features=3),
                    nn.LogSoftmax())
        
    def forward(self,input_mat):
       
        # Initial Layers
        resized_input_mat = self.reshape(input_mat)
        Inception_out_1 = self.block_i(resized_input_mat)
#         Inception_out_2 = self.block(Inception_out_1)
#         Inception_out_3 = self.block(Inception_out_2)
#         MaxPool_out_1 = self.MPoolM(Inception_out_1)
        # Auxillary 1
#         aux_1 = self.alt_out(Inception_out_1)
#         print(MaxPool_out_1.shape)
#         print(Inception_out_1.shape)
        aux_1 = self.auxillary_out(Inception_out_1)

#         # Deep Blocks 1
        Inception_out_2 = self.block(Inception_out_1)
#         Inception_out_5 = self.block(Inception_out_4)
# #         MaxPool_out_2 = self.MPoolM(Inception_out_2)
#         # Auxillary 2
        aux_2 = self.auxillary_out(Inception_out_2)

        
# #         print(Inception_out_2.shape)

#         # Deep Blocks 2
#         Inception_out_3 = self.block(MaxPool_out_2)
#         # Final Out
#         main = self.auxillary_out(Inception_out_3)
# #         print(Inception_out_3.shape)

        main = aux_2
        aux_2 = aux_1
# #         aux_1 = nn.Softmax(aux_1)
# #         aux_2 = nn.Softmax(aux_2)
# #         main = nn.Softmax(main)

        out_arr = [aux_1, aux_2, main]
    
        return(out_arr)
        

In [11]:
class MSCB(nn.Module):
    def __init__(self, small_kernel, medium_kernel, large_kernel, num_filters):
        super(MSCB, self).__init__()
        self.name = "MSCB"

        # Define Small Path
        self.convS = nn.Conv1d(in_channels = 56, out_channels = num_filters, kernel_size = small_kernel, padding = 'same')
        self.MPoolS = nn.MaxPool1d(kernel_size = small_kernel, stride = 5, padding = int(small_kernel/2 - 1))
        
        # Define Medium Path
        self.convM = nn.Conv1d(in_channels = 56, out_channels = num_filters, kernel_size = medium_kernel, padding = 'same')
        self.MPoolM = nn.MaxPool1d(kernel_size = medium_kernel, stride = 5, padding = int(medium_kernel/2 - 1))
        
        # Define Large Path
        self.convL = nn.Conv1d(in_channels = 56, out_channels = num_filters, kernel_size = large_kernel, padding = 'same')
        self.MPoolL = nn.MaxPool1d(kernel_size = large_kernel, stride = 5, padding = int(large_kernel/2 - 1))
        
        #
        self.MPool = nn.MaxPool1d(kernel_size = 3, stride = 5)
        
        #
        self.auxMPool = nn.MaxPool1d(kernel_size = 10, stride = 10)
#         self.avgpool = nn.AdaptiveAvgPool1d(12)
        
        self.auxFC = nn.Linear(in_features = 9600, out_features = 3840)
        
        self.conv = nn.Conv1d(in_channels = 56, out_channels = 128, kernel_size = 24, padding = 'same')
        
        #
        self.conv2 = nn.Conv1d(in_channels = 3*num_filters + 128, out_channels = 64, kernel_size = 112, padding = 'same')
        self.MPool2 = nn.MaxPool1d(kernel_size = 3, stride = 5)
        self.conv3 = nn.Conv1d(in_channels = 64, out_channels = 64, kernel_size = 25, padding = 'same')
        self.fc1 = nn.Linear(in_features = 3840, out_features = 400)
        self.Dropout = nn.Dropout(0.5)
        self.fc2 = nn.Linear(in_features = 400, out_features = 1024)
        self.fc3 = nn.Linear(in_features = 1024, out_features = 3)

    def forward(self, x):
        # Feature Learning Head
        
        x = torch.moveaxis(x,2,1)
        x = torch.tensor(x, dtype=torch.float32)

        x_S = self.MPoolS(F.relu(self.convS(x)))
        x_M = self.MPoolM(F.relu(self.convM(x)))
        x_L = self.MPoolL(F.relu(self.convL(x)))
        x_O = F.relu(self.conv(self.MPool(x)))
        
#         x_aux = x_aux.view(-1,3840)
        
        #Classification Head
#         x_aux = F.relu(self.fc1((x_aux)))
#         x_aux = self.Dropout(x_aux)
#         x_aux = F.relu(self.fc2(x_aux))
#         x_aux = self.fc3(x_aux)
        
        
#         print(x_S.shape)
#         print(x_M.shape)
#         print(x_L.shape)
#         print(x_O.shape)
        
        #
        x = torch.cat((x_S,x_M,x_L,x_O),1)
        
        
        x_aux = self.auxMPool(x)
#         print(x_aux.shape)
        x_aux = x_aux.view(-1,9600)
#         print(x_aux.shape)
        x_aux = self.auxFC(x_aux)
        #Classification Head
#         x_aux = x_aux.view(-1,3840)
        x_aux = F.relu(self.fc1((x_aux)))
        x_aux = self.Dropout(x_aux)
        x_aux = F.relu(self.fc2(x_aux))
        x_aux = self.fc3(x_aux)
#         x_aux = F.softmax(x_aux)

        x = self.MPool2(F.relu(self.conv2(x)))
        x = F.relu(self.conv3(x))
        
        # Flattening
        x = x.view(-1,3840)
#         print(x.shape)

        #Classification Head
        x = F.relu(self.fc1((x)))
        x = self.Dropout(x)
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
#         x = F.softmax(x)
        
        return (x, x_aux)



In [12]:
class Inception_with_auxillary(nn.Module):
    def __init__(self, n_filters=32, kernel_sizes=[9,19,39], bottleneck_channels=32, GAPoutput_size=12, use_residual=True):
        super(Inception_with_auxillary, self).__init__()
        
        self.reshape = Reshape(out_shape=(56,1500))
        self.block_i = InceptionBlock(
            in_channels=56, 
            n_filters = n_filters, 
            kernel_sizes = kernel_sizes,
            bottleneck_channels = bottleneck_channels,
            use_residual = use_residual,
            activation=nn.ReLU())
        
        self.block = InceptionBlock(
            in_channels=n_filters*4, 
            n_filters = n_filters, 
            kernel_sizes = kernel_sizes,
            bottleneck_channels = bottleneck_channels,
            use_residual = use_residual,
            activation=nn.ReLU())

        self.MPoolM = nn.MaxPool1d(kernel_size = kernel_sizes[1], stride = 5, padding = int(kernel_sizes[1]/2 - 1)),

        
        self.alt_out = nn.Sequential(
            nn.MaxPool1d(kernel_size = 3, stride = 5),
            nn.Conv1d(in_channels = n_filters*4, out_channels = 64, kernel_size = 112, padding = 'same'),
            nn.MaxPool1d(kernel_size = 3, stride = 5),
            nn.Flatten(),
            nn.Linear(in_features = 3840, out_features = 400),
            nn.Dropout(0.5),
            nn.Linear(in_features = 400, out_features = 1024),
            nn.Linear(in_features = 1024, out_features = 3),
            nn.LogSoftmax(-1))
        
        
        self.auxillary_out = nn.Sequential(
            
                    nn.AdaptiveAvgPool1d(GAPoutput_size),
                    Flatten(out_features=n_filters*4*GAPoutput_size),
#                     nn.Dropout(0.5),
                    nn.Linear(in_features=4*n_filters*GAPoutput_size, out_features=3))
        
    def forward(self,input_mat):
       
        # Initial Layers
        resized_input_mat = self.reshape(input_mat)
        Inception_out_1 = self.block_i(resized_input_mat)
#         Inception_out_2 = self.block(Inception_out_1)
#         Inception_out_3 = self.block(Inception_out_2)
#         MaxPool_out_1 = self.MPoolM(Inception_out_1)
        # Auxillary 1
#         aux_1 = self.alt_out(Inception_out_1)
#         print(MaxPool_out_1.shape)
#         print(Inception_out_1.shape)
        main = self.auxillary_out(Inception_out_1)

# #         # Deep Blocks 1
#         Inception_out_2 = self.block(Inception_out_1)
# #         Inception_out_5 = self.block(Inception_out_4)
# # #         MaxPool_out_2 = self.MPoolM(Inception_out_2)
# #         # Auxillary 2
#         aux_2 = self.auxillary_out(Inception_out_2)

        
# # #         print(Inception_out_2.shape)

# #         # Deep Blocks 2
# #         Inception_out_3 = self.block(MaxPool_out_2)
# #         # Final Out
# #         main = self.auxillary_out(Inception_out_3)
# # #         print(Inception_out_3.shape)

#         main = aux_2
#         aux_2 = aux_1
# # #         aux_1 = nn.Softmax(aux_1)
# # #         aux_2 = nn.Softmax(aux_2)
# # #         main = nn.Softmax(main)

#         out_arr = [aux_1, aux_2, main]
    
        return(main)
        

In [13]:
def cf_mat_gen(net, test_set):
    guesses = []
    labels = []
    
    for i in test_set:
        batch = (i[0])
        num,__,_ = batch.shape

        label_batch = (i[1])
        for j in range(num):

            label = np.array(label_batch)[j]

            sample = batch[j,:,:].unsqueeze(0)
            sample = sample.cuda()
            
            
            net.eval()
            sample = sample.float()
            outputs = (net(sample))

            outputs = outputs[0]
            outputs = torch.Tensor.cpu(outputs)

            np_out = outputs.detach().numpy()[0]
            guess = np.argmax(np_out, axis=0)
            guesses.append(guess)
            labels.append(label)
            
    cf_mat = confusion_matrix(guesses, labels)
    
    return cf_mat

In [14]:
n_filters = 64
learning_rate = 0.001
num_epochs = 400
use_cuda = True
early_stop = 3
batch_size = 32
small_kernel = 1
medium_kernel = 3
large_kernel = 5
min_delta = -0.025
min_epochs = 30
num_workers = 8

In [15]:
# n_filters=32
# kernel_sizes=[9,19,39]
# bottleneck_channels=32
# GAPoutput_size=12
# use_residual=True

In [16]:
class PositionalEncoding(nn.Module):

    def __init__(self, seq_len, d_model, dropout = 0.1):
        super(PositionalEncoding, self).__init__()
        max_len = max(5000, seq_len)
        self.dropout = nn.Dropout(p=dropout)
        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-math.log(10000.0) / d_model))
        pe[:, 0::2] = torch.sin(position * div_term)
        
        if d_model % 2 == 0:
            pe[:, 1::2] = torch.cos(position * div_term)
        else:
            pe[:, 1::2] = torch.cos(position * div_term)[: , 0 : -1]
        
        pe = pe.unsqueeze(0).transpose(0, 1)
        self.register_buffer('pe', pe)

    # Input: seq_len x batch_size x dim, Output: seq_len, batch_size, dim
    def forward(self, x):
        x = x + self.pe[:x.size(0), :]
        return self.dropout(x)
    
    
    
class Permute(torch.nn.Module):
    def forward(self, x):
        return x.permute(1, 0)
    
    

class MultitaskTransformerModel(nn.Module):

    def __init__(self, task_type, device, nclasses, seq_len, batch, input_size, emb_size, nhead, nhid, nhid_tar, nhid_task, nlayers, dropout = 0.1):
        super(MultitaskTransformerModel, self).__init__()
        # from torch.nn import TransformerEncoder, TransformerEncoderLayer
        
        self.trunk_net = nn.Sequential(
            nn.Linear(input_size, emb_size),
#             nn.BatchNorm1d(batch),
            nn.LayerNorm(emb_size),
            PositionalEncoding(seq_len, emb_size, dropout),
            nn.LayerNorm(emb_size),
#             nn.BatchNorm1d(batch)
        )
        
        # encoder_layers = transformer_encoder_class.TransformerEncoderLayer(emb_size, nhead, nhid, out_channel, filter_height, filter_width, dropout)
        # encoder_layers = TransformerEncoderLayer(emb_size, nhead, nhid, dropout)
        # self.transformer_encoder = TransformerEncoder(encoder_layers, nlayers)
        
        encoder_layers = TAR_transformer.TransformerEncoderLayer(emb_size, nhead, nhid, dropout)
        self.transformer_encoder = TAR_transformer.TransformerEncoder(encoder_layers, nlayers, device)
        
        self.batch_norm = nn.BatchNorm1d(batch)
        
        self.layer_norm = nn.LayerNorm(emb_size)
        
        # Task-aware Reconstruction Layers
        self.tar_net = nn.Sequential(
            nn.Linear(emb_size, nhid_tar),
            nn.BatchNorm1d(batch),
            nn.Linear(nhid_tar, nhid_tar),
            nn.BatchNorm1d(batch),
            nn.Linear(nhid_tar, input_size),
        )

        if task_type == 'classification':
            # Classification Layers
            self.class_net = nn.Sequential(
                nn.Linear(emb_size, nhid_task),
                nn.ReLU(),
                Permute(),
#                 nn.BatchNorm1d(batch),
                Permute(),
                nn.Dropout(p = 0.3),
                nn.Linear(nhid_task, nhid_task),
                nn.ReLU(),
                Permute(),
#                 nn.BatchNorm1d(batch),
                Permute(),
                nn.Dropout(p = 0.3),
                nn.Linear(nhid_task, nclasses)
            )
        else:
            # Regression Layers
            self.reg_net = nn.Sequential(
                nn.Linear(emb_size, nhid_task),
                nn.ReLU(),
                Permute(),
                nn.BatchNorm1d(batch),
                Permute(),
                nn.Linear(nhid_task, nhid_task),
                nn.ReLU(),
                Permute(),
                nn.BatchNorm1d(batch),
                Permute(),
                nn.Linear(nhid_task, 1),
            )
            

        
    def forward(self, x):
        x = torch.tensor(x, dtype=torch.float32)
        x = self.trunk_net(x.permute(2, 0, 1))
        x, attn = self.transformer_encoder(x)
#         x = self.batch_norm(x)
        # x : seq_len x batch x emb_size
        output = self.class_net(x[-1])
        return output, attn


In [18]:
class MSCB(nn.Module):
    def __init__(self, small_kernel, medium_kernel, large_kernel, num_filters):
        super(MSCB, self).__init__()
        self.name = "MSCB"

        # Define Small Path
        self.convS = nn.Conv1d(in_channels = 56, out_channels = num_filters, kernel_size = small_kernel, padding = 'same')
        self.MPoolS = nn.MaxPool1d(kernel_size = small_kernel, stride = 5, padding = int(small_kernel/2 - 1))
        
        # Define Medium Path
        self.convM = nn.Conv1d(in_channels = 56, out_channels = num_filters, kernel_size = medium_kernel, padding = 'same')
        self.MPoolM = nn.MaxPool1d(kernel_size = medium_kernel, stride = 5, padding = int(medium_kernel/2 - 1))
        
        # Define Large Path
        self.convL = nn.Conv1d(in_channels = 56, out_channels = num_filters, kernel_size = large_kernel, padding = 'same')
        self.MPoolL = nn.MaxPool1d(kernel_size = large_kernel, stride = 5, padding = int(large_kernel/2 - 1))
        
        # MPool first
        self.MPool = nn.MaxPool1d(kernel_size = 3, stride = 5)
        self.conv = nn.Conv1d(in_channels = 56, out_channels = 128, kernel_size = 24, padding = 'same')
        
        # Post Concatenation
        self.conv2 = nn.Conv1d(in_channels = 3*num_filters + 128, out_channels = 64, kernel_size = 112, padding = 'same')
        self.MPool2 = nn.MaxPool1d(kernel_size = 3, stride = 5)
        self.fc1 = nn.Linear(in_features = 256, out_features = 400)
        self.Dropout = nn.Dropout(0.5)
        self.fc2 = nn.Linear(in_features = 400, out_features = 1024)
        self.fc3 = nn.Linear(in_features = 1024, out_features = 3)

    def forward(self, x):
        ### Feature Learning Head
        
        # Reshape Tensor
        x = torch.moveaxis(x,2,1)
        x = torch.tensor(x, dtype=torch.float32)

        # Parrallel Convolution Pathways
        x_S = self.MPoolS(F.relu(self.convS(x)))
        x_M = self.MPoolM(F.relu(self.convM(x)))
        x_L = self.MPoolL(F.relu(self.convL(x)))
        x_O = F.relu(self.conv(self.MPool(x)))
        
        # Post Concatenation
        x = torch.cat((x_S,x_M,x_L,x_O),1)
        x = self.MPool2(F.relu(self.conv2(x)))
        
        # Flattening
        x = x.view(-1,256)

        #Classification Head
        x = F.relu(self.fc1((x)))
        x = self.Dropout(x)
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

In [19]:
task_type = "classification"
device = "cuda"
nclasses = 3
seq_len = 1500
batch = 64
input_size = 1500
emb_size = 512
nhead = 8
nhid = 64
nhid_tar = 1024
nhid_task = 128
nlayers = 2


learning_rate = 0.001
num_epochs = 80
use_cuda = True
early_stop = 3
min_delta = -0.025
min_epochs = 30
num_workers = 8

n_filters = 64
small_kernel = 1
medium_kernel = 3
large_kernel = 5

In [20]:
count2 = 0
for j in [2,3,4,5,6,7,8,9,10]:
    rat = "Rat " + str(j)
#     print(rat)

    base_dir = "M:\Peripheral Nerve Studies\MCC Projects\Aseem G\Models\Pytorch\Data\Spike 100\Minimum Dataset\\" + rat + "\\"
    fold1_dir = base_dir + "Fold1"
    fold2_dir = base_dir + "Fold2"
    fold3_dir = base_dir + "Fold3"
    test_dir = base_dir + "Test"
    train_set_1, valid_set_1, train_set_2, valid_set_2, train_set_3, valid_set_3, test_set = three_fold_cross_sets(base_dir,fold1_dir,fold2_dir,fold3_dir,test_dir, batch_size = 32)

    # [25, "final", "final"]
    epochs_list = [["final","final","final"], ["final","final","final"], ["final","final","final"], ["final","final","final"], ["final","final","final"],["final","final","final"],["final","final","final"],["final","final","final"],["final","final","final"],["final","final","final"],["final","final","final"],["final","final","final"]]
    print(len(epochs_list))
    epochs = epochs_list[count2]
    print(epochs)
    
    count2 += 1
    
    small_kernel = 1
    medium_kernel = 3
    large_kernel = 5

    one_CNN = MSCB(small_kernel, medium_kernel, large_kernel, n_filters)
    one_CNN = one_CNN.cuda()
    one_CNN = one_CNN.eval()
    count = 1
    
    cfmats = []
    t_cfmats = []

    for i in epochs:
#         print(rat)
        path = "M:\Peripheral Nerve Studies\MCC Projects\Aseem G\Models\Pytorch\model_checkpoints\\" + rat + "\\March 20 MSCB 100-2\\"
        weights = rat + "kernels_1_3_5Fold_" + str(count) + "_epochno_" + str(i)
        if i == "final":
            weights = rat + "kernels_1_3_5Fold_" + str(count) + "_final"

        print("weights")
        print(weights)
        weights_path = path + weights
#         print(weights_path)
#         state = torch.load(weights_path, map_location=torch.device('cpu') )
        state = torch.load(weights_path, map_location=torch.device('cuda'))
        one_CNN.load_state_dict(state)

        valid_set = [valid_set_1, valid_set_2, valid_set_3][count - 1]

        print(valid_set)
        
#         valid_set = valid_set.cuda()
#         test_set = test_set.cuda()

        cf_mat = cf_mat_gen(one_CNN, valid_set)
        f = open(path + "Best.txt", "a")
        f.write("\nFold " + str(count) + "\n")
        f.write("Best Epoch =" + str(i) + "\n")
        f.write(str(cf_mat))
        f.write("\nValid Accuracy: " + str(accuracy(cf_mat)) + "%\n")
        f.write("Valid Macro F1 Score: " + str(macro_f1(cf_mat)))
        f.write("\n")
        f.close()
        
        
        cfmats.append(accuracy(cf_mat))
        cfmats.append(macro_f1(cf_mat))
        
        f = open(path + "Best.txt", "a")
        cf_mat = cf_mat_gen(one_CNN, test_set)
        f.write(str(cf_mat))
        f.write("\nTest Accuracy: " + str(accuracy(cf_mat)) + "%\n")
        f.write("Test Macro F1 Score: " + str(macro_f1(cf_mat)))
        f.write("\n")
        f.close()
        count += 1
        
        t_cfmats.append(accuracy(cf_mat))
        t_cfmats.append(macro_f1(cf_mat))
    
    string = ""
    for i in cfmats:
        if i <= 1:
            string += str(i) + "\t"
        elif i > 1:
            string += str(i) + "%\t"

    f = open(path + "Best.txt", "a")
    f.write("\nValidation Accuracy\tValid Macro F1 Score\tValidation Accuracy\tValid Macro F1 Score\tValidation Accuracy\tValid Macro F1 Score\n")
    f.write(string)
    f.close()
    
    string = ""
    for i in t_cfmats:
        if i <= 1:
            string += str(i) + "\t"
        elif i > 1:
            string += str(i) + "%\t"

    f = open(path + "Best.txt", "a")
    f.write("\nTest Accuracy\tTest Macro F1 Score\tTest Accuracy\tTest Macro F1 Score\tTest Accuracy\tTest Macro F1 Score\n")
    f.write(string)
    f.close()



12
['final', 'final', 'final']
weights
Rat 2kernels_1_3_5Fold_1_final


C:\Users\GillA\AppData\Local\Temp\ipykernel_25728\2294328818.py:35: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  x = torch.tensor(x, dtype=torch.float32)
C:\Users\GillA\.conda\envs\Aseem\lib\site-packages\torch\nn\modules\conv.py:303: UserWarning: Using padding='same' with even kernel lengths and odd dilation may require a zero-padded copy of the input be created (Triggered internally at  C:\cb\pytorch_1000000000000\work\aten\src\ATen\native\Convolution.cpp:883.)
  return F.conv1d(input, weight, bias, self.stride,
C:\Users\GillA\AppData\Local\Temp\ipykernel_25728\2758336073.py:11: RuntimeWarning: invalid value encountered in longlong_scalars
  return(num/den)
C:\Users\GillA\AppData\Local\Temp\ipykernel_25728\2294328818.py:35: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceT

weights
Rat 2kernels_1_3_5Fold_2_final


C:\Users\GillA\AppData\Local\Temp\ipykernel_25728\2294328818.py:35: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  x = torch.tensor(x, dtype=torch.float32)
C:\Users\GillA\AppData\Local\Temp\ipykernel_25728\2758336073.py:11: RuntimeWarning: invalid value encountered in longlong_scalars
  return(num/den)
C:\Users\GillA\AppData\Local\Temp\ipykernel_25728\2294328818.py:35: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  x = torch.tensor(x, dtype=torch.float32)
C:\Users\GillA\AppData\Local\Temp\ipykernel_25728\2758336073.py:11: RuntimeWarning: invalid value encountered in longlong_scalars
  return(num/den)


weights
Rat 2kernels_1_3_5Fold_3_final


...

C:\Users\GillA\AppData\Local\Temp\ipykernel_25728\2294328818.py:35: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  x = torch.tensor(x, dtype=torch.float32)
C:\Users\GillA\AppData\Local\Temp\ipykernel_25728\2758336073.py:11: RuntimeWarning: invalid value encountered in longlong_scalars
  return(num/den)
C:\Users\GillA\AppData\Local\Temp\ipykernel_25728\2294328818.py:35: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  x = torch.tensor(x, dtype=torch.float32)
C:\Users\GillA\AppData\Local\Temp\ipykernel_25728\2758336073.py:11: RuntimeWarning: invalid value encountered in longlong_scalars
  return(num/den)


weights
Rat 10kernels_1_3_5Fold_2_final


C:\Users\GillA\AppData\Local\Temp\ipykernel_25728\2294328818.py:35: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  x = torch.tensor(x, dtype=torch.float32)
C:\Users\GillA\AppData\Local\Temp\ipykernel_25728\2758336073.py:11: RuntimeWarning: invalid value encountered in longlong_scalars
  return(num/den)
C:\Users\GillA\AppData\Local\Temp\ipykernel_25728\2294328818.py:35: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  x = torch.tensor(x, dtype=torch.float32)
C:\Users\GillA\AppData\Local\Temp\ipykernel_25728\2758336073.py:11: RuntimeWarning: invalid value encountered in longlong_scalars
  return(num/den)


weights
Rat 10kernels_1_3_5Fold_3_final


C:\Users\GillA\AppData\Local\Temp\ipykernel_25728\2294328818.py:35: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  x = torch.tensor(x, dtype=torch.float32)
C:\Users\GillA\AppData\Local\Temp\ipykernel_25728\2758336073.py:11: RuntimeWarning: invalid value encountered in longlong_scalars
  return(num/den)
C:\Users\GillA\AppData\Local\Temp\ipykernel_25728\2294328818.py:35: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  x = torch.tensor(x, dtype=torch.float32)
C:\Users\GillA\AppData\Local\Temp\ipykernel_25728\2758336073.py:11: RuntimeWarning: invalid value encountered in longlong_scalars
  return(num/den)


In [53]:
one_CNN = MSCB(n_filters, small_kernel, medium_kernel, large_kernel)

In [54]:
one_CNN

MSCB(
  (convS): Conv1d(56, 5, kernel_size=(64,), stride=(1,), padding=same)
  (MPoolS): MaxPool1d(kernel_size=64, stride=5, padding=31, dilation=1, ceil_mode=False)
  (convM): Conv1d(56, 5, kernel_size=(1,), stride=(1,), padding=same)
  (MPoolM): MaxPool1d(kernel_size=1, stride=5, padding=0, dilation=1, ceil_mode=False)
  (convL): Conv1d(56, 5, kernel_size=(3,), stride=(1,), padding=same)
  (MPoolL): MaxPool1d(kernel_size=3, stride=5, padding=0, dilation=1, ceil_mode=False)
  (MPool): MaxPool1d(kernel_size=3, stride=5, padding=0, dilation=1, ceil_mode=False)
  (conv): Conv1d(56, 128, kernel_size=(24,), stride=(1,), padding=same)
  (conv2): Conv1d(143, 64, kernel_size=(112,), stride=(1,), padding=same)
  (MPool2): MaxPool1d(kernel_size=3, stride=5, padding=0, dilation=1, ceil_mode=False)
  (fc1): Linear(in_features=3840, out_features=400, bias=True)
  (Dropout): Dropout(p=0.5, inplace=False)
  (fc2): Linear(in_features=400, out_features=1024, bias=True)
  (fc3): Linear(in_features=1024

In [ ]:
state

In [21]:
cf_mat

array([[638,   1,   1],
       [266, 865,   1],
       [  0,  38, 902]], dtype=int64)